In [1]:
!pip install semantic-link-labs

StatementMeta(, bad056f4-0aa5-4743-9663-648699cc5fa4, 3, Finished, Available, Finished)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.4/819.4 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 99.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.8/217.8 kB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.0/43.0 kB 9.9 MB/s eta 0:00:00
  Attempting uninstall: azure-core
    Found existing installation: azure-core 1.30.2
    Uninstalling azure-core-1.30.2:
      Successfully uninstalled azure-core-1.30.2
  Attempting uninstall: semantic-link-sempy
    Found existing installation: semantic-link-sempy 0.11.0
    Uninstalling semantic-link-sempy-0.11.0:
      Successfully uninstalled semantic-link-sempy-0.11.0
ERROR: pip's dependency resolver does not currently take into acco

In [2]:
spark.conf.set("spark.sql.legacy.timeParserPolicy" , "LEGACY")

StatementMeta(, bad056f4-0aa5-4743-9663-648699cc5fa4, 4, Finished, Available, Finished)

**Variables for service principal and the Capacity Metrics Workspace Name and Dataset Name**

In [3]:
capacity_metrics_ws_name = "Microsoft Fabric Capacity Metrics"
capacity_metrics_target_dataset_name = "Fabric Capacity Metrics"

tenant_id = ""
subscriptionid = ""
client_id = ""
client_secret = ""

StatementMeta(, bad056f4-0aa5-4743-9663-648699cc5fa4, 5, Finished, Available, Finished)

**Helper functions to format column names**

In [4]:
import sempy.fabric as fabric
from sempy.fabric.exceptions import FabricHTTPException
import pandas as pd
import re
from pyspark.sql.functions import col
from pyspark.sql import functions as F
from pyspark.sql.functions import lit
from pyspark.sql.types import StringType

def sanitize_column_name(name: str) -> str:
    # Replace invalid characters with underscores, keep alphanumeric and underscores
    sanitized = re.sub(r'[^a-zA-Z0-9_]', '_', name)
    # Ensure name doesn't start with a digit
    if sanitized and sanitized[0].isdigit():
        sanitized = f"col_{sanitized}"
    return sanitized.lower()

def format_column_names(df):   
    formatted_col_list = [sanitize_column_name(col_name) for col_name in df.columns]
    formatted_col_list = [col.rstrip('_') for col in formatted_col_list]
    df = df.toDF(*formatted_col_list)
    return df



StatementMeta(, bad056f4-0aa5-4743-9663-648699cc5fa4, 6, Finished, Available, Finished)

**1.0 The code fetches service principal from Azure Key Vault.**

It gets an access token once, then uses it to call the Key Vault REST API for each secret.
It includes retry logic for transient errors and returns a dictionary of secret names and values.

In [5]:
# Standard library & third-party imports
import requests                              # HTTP client for calling REST APIs (used here for Key Vault)
import json                                  # JSON serialization/deserialization
import os                                    # OS utilities (env vars, paths)
from datetime import datetime, timedelta     # Date/time utilities
import pandas as pd                          # Data processing (not used in this snippet)
import sempy.fabric as fabric                # Microsoft Fabric / semantic tooling (imported but not used below)
from typing import Iterable, Dict, Optional  # Type hints



# -----------------------------
# Key Vault Secret Fetch Helper
# -----------------------------
def get_kv_secrets(
    kv_name: str,
    secret_names: Iterable[str],
    api_version: str = "7.3",
    timeout_sec: int = 30,
    max_retries: int = 2,
    backoff_sec: float = 0.5,
) -> Dict[str, Optional[str]]:
    """
    Fetch one or more secrets from Azure Key Vault using the notebook's execution identity.
    Uses notebookutils.credentials.getToken('https://vault.azure.net') once and reuses it
    for all secret calls.

    Args:
        kv_name:                Key Vault name (e.g., 'kvrbacpocsk')
        secret_names:           Iterable of secret names to fetch (e.g., ['TENANTID', 'subscriptionid'])
        api_version:            Key Vault API version (default '7.3')
        timeout_sec:            Per-call HTTP timeout
        max_retries:            Number of retries on transient 5xx/429 errors
        backoff_sec:            Backoff seconds between retries

    Returns:
        Dict mapping secret name -> secret value (or None if not found/failed).
    """
    # 1) Acquire a bearer token for Key Vault using the Spark notebook identity.
    #    This relies on Synapse/Fabric's notebookutils to obtain an AAD token scoped to Key Vault.
    token = notebookutils.credentials.getToken("https://vault.azure.net")

    # Standard auth + content headers; we reuse these for all requests
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json",
    }

    # Base URL for the target Key Vault (data-plane endpoint)
    base_url = f"https://{kv_name}.vault.azure.net"

    # Small inner helper to GET with basic retry/backoff handling
    def _get_with_retry(url: str) -> Optional[dict]:
        attempt = 0
        while True:
            try:
                # Make the HTTP GET call with a per-call timeout
                resp = requests.get(url, headers=headers, timeout=timeout_sec)

                # Retry on common transient statuses (throttling/temporary failures)
                if resp.status_code in (429, 500, 502, 503, 504):
                    if attempt < max_retries:
                        attempt += 1
                        import time
                        time.sleep(backoff_sec * attempt)  # Linear backoff by attempt count
                        continue  # Try again
                # Raise for any non-success HTTP status codes (will enter except)
                resp.raise_for_status()

                # On success, return parsed JSON (Key Vault returns JSON for secrets)
                return resp.json()

            except requests.exceptions.HTTPError as e:
                # If Key Vault returns 404, the secret doesn't exist — treat as a clean miss
                if getattr(e.response, "status_code", None) == 404:
                    return None
                # For other HTTP errors, propagate the exception up to the caller
                raise

            except requests.exceptions.RequestException:
                # Network or other request-layer issues — retry if budget remains
                if attempt < max_retries:
                    attempt += 1
                    import time
                    time.sleep(backoff_sec * attempt)
                    continue
                # Retries exhausted — re-raise to signal failure
                raise

    # Aggregate results: name -> value (or None if not found/failed)
    results: Dict[str, Optional[str]] = {}
    for name in secret_names:
        # Build the per-secret URL; Key Vault "secrets" data-plane API
        url = f"{base_url}/secrets/{name}?api-version={api_version}"

        # Attempt to fetch the secret JSON (may be None if 404 or repeated failures)
        data = _get_with_retry(url)

        # Key Vault returns {"value": "<secret>"} on success; otherwise None for misses/failures
        results[name] = data.get("value") if data else None

    # Return a flat dict mapping requested secret names to their values (or None)
    return results

StatementMeta(, bad056f4-0aa5-4743-9663-648699cc5fa4, 7, Finished, Available, Finished)

**2.0 This code looks up the workspace and dataset IDs for the Fabric Capacity Metrics dataset.**

It uses the Fabric client to find the workspace by name, then tries to resolve the dataset ID directly.
If that fails, it lists all datasets in the workspace and searches for the requested one.
Finally, it returns the workspace name, workspace ID, dataset name, and dataset ID (or None if not found).

In [6]:
import pandas as pd
from sempy.fabric import set_service_principal
import sempy.fabric as fabric

def extract_metrics_workspace_details():

    with set_service_principal(tenant_id=tenant_id, client_id=client_id, client_secret=client_secret):

        # 1) Resolve workspace by name (case/whitespace normalized)
        ws_df = fabric.list_workspaces()
        ws_match = ws_df[
            ws_df["Name"].str.strip().str.casefold() == capacity_metrics_ws_name.strip().casefold()
        ]

        if ws_match.empty:
            raise ValueError(f"Workspace '{capacity_metrics_ws_name}' not found.")
        
        target_workspace_id = ws_match.iloc[0]["Id"]

        # 2) List datasets (semantic models) in this workspace
        ds_df = fabric.list_items(workspace=target_workspace_id, item_type="semanticModel")

        # display(ds_df)

        if ds_df is None or ds_df.empty:
            raise ValueError(f"No datasets found in workspace '{capacity_metrics_ws_name}'")

        # Normalize DisplayName for comparison (handles trailing CR/LF)
        ds_df["NormalizedName"] = ds_df["Display Name"].str.strip().str.replace("\r","").str.replace("\n","").str.casefold()

    target_normalized = capacity_metrics_target_dataset_name.strip().casefold()

    ds_match = ds_df[ds_df["NormalizedName"] == target_normalized]

    if ds_match.empty:
        raise ValueError(f"Dataset '{capacity_metrics_target_dataset_name}' not found in workspace '{capacity_metrics_ws_name}'")

    target_dataset_id = ds_match.iloc[0]["Id"]
    return capacity_metrics_ws_name, target_workspace_id, capacity_metrics_target_dataset_name, target_dataset_id


StatementMeta(, bad056f4-0aa5-4743-9663-648699cc5fa4, 8, Finished, Available, Finished)

**3.0 This function triggers a dataset (semantic model) refresh in Fabric/Power BI and then polls the refresh status until it finishes.**

It starts the refresh, identifies where to check progress, repeatedly checks the refresh state, and returns the final result (success or failure).
If polling never reaches a terminal state, it times out.

In [7]:

import time
import sempy.fabric as fabric

def refresh_metrics_dataset(
    workspace_id: str,
    dataset_id: str,
    *,
    refresh_type: str = "Full",          # "Full", "Calculate", "DataOnly", "ClearValues", "Automatic", "Defragment"
    poll_interval_sec: int = 15,         # how often to poll
    max_attempts: int = 60               # 15s * 60 = 15 minutes total wait
) -> dict:
    """
    Triggers a Power BI/Fabric semantic model (dataset) refresh and polls until terminal state.
    Returns the final refresh record (dict). Raises RuntimeError/TimeoutError on issues.
    """

    with set_service_principal(tenant_id=tenant_id, client_id=client_id, client_secret=client_secret):
        frc = fabric.FabricRestClient()

    # 1) Trigger workspace-scoped refresh

    trigger_path = f"https://api.powerbi.com/v1.0/myorg/datasets/{dataset_id}/refreshes"
    body = {
        "type": refresh_type
    }

    resp = frc.post(trigger_path, json=body)
    if resp.status_code != 202:
        raise RuntimeError(f"Failed to trigger refresh: {resp.status_code} {resp.text}")

    # 2) Prefer Location header to poll the specific refresh operation
    location = resp.headers.get("Location")
    request_id = resp.headers.get("RequestId") or resp.headers.get("requestid")

    if location:
        # Location is a full URL. Convert to a relative path for FabricRestClient (which wants paths).
        # Example Location: https://api.powerbi.com/v1.0/myorg/groups/{g}/datasets/{d}/refreshes/{rid}
        poll_path = location.split(".com")[-1]  # keep everything after domain
    elif request_id:
        poll_path = f"/groups/{workspace_id}/datasets/{dataset_id}/refreshes/{request_id}"
    else:
        # Fallback: poll the collection and read latest entry
        poll_path = f"/groups/{workspace_id}/datasets/{dataset_id}/refreshes"

    # 3) Poll until terminal status
    for attempt in range(max_attempts):
        r = frc.get(poll_path)
        if r.status_code not in (200, 202):
            raise RuntimeError(f"Polling error: {r.status_code} {r.text}")

        data = r.json()
        # If polling a single refresh, JSON is an object; if polling the collection, JSON has 'value'
        refresh = data if "status" in data else (data.get("value", [{}])[0])
        status = (refresh or {}).get("status")
        print(f"[{attempt}] status={status}")

        if status in ("Completed", "Failed", "Cancelled"):
            if status == "Completed":
                print("Refresh succeeded.")
            else:
                print(f"Refresh finished with status: {status}")
                # When available, serviceExceptionJson gives richer details
                print(refresh.get("serviceExceptionJson") or refresh)
            return refresh

        time.sleep(poll_interval_sec)

    raise TimeoutError("Polling timed out. Check refresh history in the Power BI service.")


StatementMeta(, bad056f4-0aa5-4743-9663-648699cc5fa4, 9, Finished, Available, Finished)

**4.0 This code updates two mashup parameters (DefaultCapacityID and RegionName) on a Fabric/Power BI dataset and can optionally verify that the parameters exist first.**

After updating, it can automatically trigger a dataset refresh to apply the new values.
It uses the Fabric REST client and raises clear errors if parameter validation or HTTP calls fail.

In [8]:
import sempy.fabric as fabric
from sempy.fabric.exceptions import FabricHTTPException


def update_metrics_dataset(
    metrics_workspace_id: str,
    metrics_dataset_id: str,
    capacity_id_to_extract_data_from: str,
    region_name: str,
    *,
    verify_parameters_exist: bool = True,
    trigger_refresh: bool = True,
) -> None:
    """
    Update two mashup parameters on a Fabric/Power BI dataset:
        - DefaultCapacityID
        - RegionName

    Then (optionally) trigger a dataset refresh.

    Args:
        metrics_workspace_id: Workspace (group) GUID containing the dataset
        metrics_dataset_id: Dataset GUID (semantic model)
        capacity_id_to_extract_data_from: New value for DefaultCapacityID (GUID)
        region_name: New value for RegionName (e.g., 'West US 3')
        verify_parameters_exist: If True, GET parameters first and fail fast if missing
        trigger_refresh: If True, call refresh_metrics_dataset(...) after update

    Raises:
        RuntimeError / FabricHTTPException on HTTP failures or parameter validation errors.
    """
    with set_service_principal(
        tenant_id=tenant_id, client_id=client_id, client_secret=client_secret
    ):
        client = fabric.FabricRestClient()

    # ----------------------------------------------------------------------
    # (Optional) Verify the two parameters exist before attempting update
    # ----------------------------------------------------------------------
    if verify_parameters_exist:
        get_params_path = (
            "https://api.powerbi.com/v1.0/myorg/groups/"
            f"{metrics_workspace_id}/datasets/{metrics_dataset_id}/parameters"
        )
        try:
            r = client.get(get_params_path)
        except FabricHTTPException as ex:
            raise RuntimeError(f"Failed to read dataset parameters: {ex}") from ex

        if r.status_code != 200:
            raise RuntimeError(f"Cannot read parameters: {r.status_code} {r.text}")

        params = r.json().get("value", [])
        names = {p.get("name") for p in params}
        required = {"DefaultCapacityID", "RegionName"}
        missing = required - names
        if missing:
            raise RuntimeError(
                f"Dataset is missing required parameters: {sorted(missing)}. "
                f"Found: {sorted(names)}"
            )

    # ----------------------------------------------------------------------
    # Build payload and POST UpdateParameters (workspace scoped)
    # ----------------------------------------------------------------------
    utc_offset = "0"
    update_path = (
        "https://api.powerbi.com/v1.0/myorg/groups/"
        f"{metrics_workspace_id}/datasets/{metrics_dataset_id}/Default.UpdateParameters"
    )
    payload = {
        "updateDetails": [
            {"name": "DefaultCapacityID", "newValue": str(capacity_id_to_extract_data_from)},
            {"name": "RegionName", "newValue": str(region_name)},
            {"name": "UTC_offset", "newValue": str(utc_offset)},
        ]
    }

    try:
        resp = client.post(update_path, json=payload)
    except FabricHTTPException as ex:
        raise RuntimeError(f"Failed to update dataset parameters: {ex}") from ex

    if resp.status_code != 200:
        raise RuntimeError(f"UpdateParameters failed: {resp.status_code} {resp.text}")

    print("✅ Parameters updated successfully.")

    # ----------------------------------------------------------------------
    # (Optional) trigger a refresh so new parameter values are applied
    # ----------------------------------------------------------------------
    if trigger_refresh:
        # assumes you have the helper defined as in our earlier step:
        #   refresh_metrics_dataset(workspace_id, dataset_id, ...)
        try:
            print("🔄 Triggering dataset refresh…")
            _ = refresh_metrics_dataset(
                workspace_id=metrics_workspace_id,
                dataset_id=metrics_dataset_id,
                refresh_type="Full",
                poll_interval_sec=15,
                max_attempts=60,
            )
        except Exception:
            # Surface but do not hide the successful parameter update
            raise RuntimeError("Parameters are not valid")


StatementMeta(, bad056f4-0aa5-4743-9663-648699cc5fa4, 10, Finished, Available, Finished)

**5.0 This code queries a Fabric/Power BI dataset table using DAX, handles pagination, cleans up the returned column names, and writes the results into a Delta table.**

It also includes helpers to sanitize column names and compute a default start date by looking up the maximum date from an existing table.

In [9]:
import sempy.fabric as fabric
from sempy.fabric.exceptions import FabricHTTPException
import pandas as pd
import re
from pyspark.sql.functions import col
from pyspark.sql import functions as F
from pyspark.sql.functions import lit
from pyspark.sql.types import StringType

def get_table_data(metrics_workspace_id, metrics_dataset_id, table_name, capacity_id):

    try:
        url = f"https://api.powerbi.com/v1.0/myorg/groups/{metrics_workspace_id}/datasets/{metrics_dataset_id}/executeQueries"


        if table_name == "Metrics By Item and Day":
            
            start_iso = get_max_or_default("metrics_by_item_and_day", "metrics_by_item_and_day_datetime")
            print(f"getting the data from start_iso = {start_iso}")

            # dax_query = {
            # "queries": [
            #     {
            #     "query": f"EVALUATE FILTER('{table_name}', '{table_name}'[Date] >= DATEVALUE(\"{start_iso}\"))  && '{table_name}'[[Capacity Id]] = \"{capacity_id}\""
            #     # "query": f"EVALUATE FILTER('{table_name}', '{table_name}'[Date] >= DATEVALUE(\"{start_iso}\"))"                
            #     }
            # ],
            # "serializerSettings": { "includeNulls": True }
            # }

            dax_query = {
                "queries": [
                    {
                        "query": f"""
            EVALUATE
            FILTER(
                'Metrics By Item and Day',
                'Metrics By Item and Day'[Date] >= DATEVALUE("{start_iso}")
                    && 'Metrics By Item and Day'[Capacity Id] = "{capacity_id}"
            )
            """
                    }
                ]
            }

        else:
            dax_query = {
                "queries": [{
                    "query":f"EVALUATE '{table_name}'"
                }],
                "serializerSettings": {"includeNulls": True}
            }

        headers = {
            "Content-Type": "application/json",

        }

        rows = []
        continuation_token = None

        while True:

            query_payload = dax_query.copy()
            if continuation_token:
                query_payload["continuationToken"] = continuation_token

            with set_service_principal(tenant_id=tenant_id, client_id=client_id, client_secret=client_secret): 
                 fabric_rest_client = fabric.FabricRestClient()

            resp = fabric_rest_client.post(url, headers=headers, json=query_payload)
            result = resp.json()
            # print(result)  
            
            # Parse rows
            new_rows = result['results'][0]['tables'][0]['rows']
            rows.extend(new_rows)

            # Check for next page
            continuation_token = result['results'][0].get('continuationToken')
            if not continuation_token:
                break

        df_metrics = pd.DataFrame(rows)
        df_metrics = spark.createDataFrame(df_metrics)
        df_metrics = format_column_names(df_metrics)
        table_name = "stg_" + sanitize_column_name(f'{table_name}')

        # Cast each void column to string
        void_cols = [col for col, dtype in df_metrics.dtypes if dtype == 'void']
        for col in void_cols:
            df_metrics = df_metrics.withColumn(col, lit(None).cast(StringType()))
        df_metrics.write.option("mergeSchema", "true").mode('overwrite').format('delta').saveAsTable(f'{table_name}')

    except Exception as e:
        print(f"An unexpected error occurred: {e}")

StatementMeta(, bad056f4-0aa5-4743-9663-648699cc5fa4, 11, Finished, Available, Finished)

In [10]:
import sempy.fabric as fabric
from sempy.fabric.exceptions import FabricHTTPException
import pandas as pd
import re
from pyspark.sql.functions import col
from pyspark.sql import functions as F
from pyspark.sql.functions import lit
from pyspark.sql.types import StringType
from pyspark.sql import functions as F

def get_max_or_default(table_name: str, column_name: str, default_value: str = "1970-01-01") -> str:
    """
    Returns (max(column) - 7 day) as 'YYYY-MM-DD' from a given table/column.
    If the table/column doesn't exist or is empty, returns default_value.
    """
    days_to_extract = 7
    # Check if table exists
    if spark.catalog.tableExists(table_name):
        df = spark.read.table(table_name)

        # Check if column exists
        if column_name in df.columns:
            # Parse column as date (handles date/timestamp/text formats)
            parsed_date = F.coalesce(
                F.col(column_name).cast("date"),
                F.to_date(F.col(column_name), "yyyy-MM-dd"),
                F.to_date(F.col(column_name), "MM/dd/yyyy"),
                F.to_date(F.col(column_name), "yyyy/MM/dd"),
                F.to_timestamp(F.col(column_name), "yyyy-MM-dd HH:mm:ss").cast("date"),
                F.to_timestamp(F.col(column_name), "yyyy-MM-dd'T'HH:mm:ss").cast("date")
            )

            # Compute max date
            max_val = df.select(F.max(parsed_date).alias("max_date")).collect()[0]["max_date"]

            if max_val is not None:
                # Subtract 1 day and return as string
                adjusted_date = spark.createDataFrame([(max_val,)], ["d"]) \
                                      .select(F.date_sub(F.col("d"), days_to_extract).alias("d_sub")) \
                                      .collect()[0]["d_sub"]
                return str(adjusted_date)
            else:
                print(f"Column '{column_name}' is empty. Using default.")
                return default_value
        else:
            print(f"Column '{column_name}' not found in table '{table_name}'. Using default.")
            return default_value
    else:
        print(f"Table '{table_name}' does not exist. Using default.")
        return default_value



StatementMeta(, bad056f4-0aa5-4743-9663-648699cc5fa4, 12, Finished, Available, Finished)

**6.0 This code checks for patterns (integers, floats, dates, timestamps, or booleans) and applies the correct Spark cast.**

It also fixes columns that Spark loads as all-null (NullType) by casting them to a real type, usually string or a type from a provided schema.

In [11]:
from pyspark.sql import functions as F
from pyspark.sql.types import (
    StringType, LongType, DoubleType, BooleanType, DateType, TimestampType, NullType
)
import re

sample_size = 1000

def looks_like(patterns, s): return any(re.fullmatch(p, s) for p in patterns)

def infer_column_type(series):
    date_patterns = [r"\d{4}-\d{2}-\d{2}", r"\d{2}/\d{2}/\d{4}", r"\d{4}/\d{2}/\d{2}"]
    ts_patterns   = [r"\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}", r"\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}"]

    # -------- Inference helpers --------
    def is_int(s): return re.fullmatch(r"-?\d+", s) is not None
    def is_float(s): return re.fullmatch(r"-?\d+(?:\.\d+)?", s) is not None
    def is_bool(s): return s.lower() in {"true", "false", "yes", "no", "0", "1"}

    cleaned = series.dropna().astype(str).str.strip()
    cleaned = cleaned[cleaned != ""]
    if cleaned.empty: return "string"
    if cleaned.apply(is_bool).all(): return "boolean"
    if cleaned.apply(is_int).all(): return "long"
    if cleaned.apply(is_float).all(): return "double"
    if cleaned.apply(lambda s: looks_like(ts_patterns, s)).all(): return "timestamp"
    if cleaned.apply(lambda s: looks_like(date_patterns, s)).all(): return "date"
    return "string"

# -------- Casting helper (strings -> typed) --------
def cast_column(col_name, target_type):
    c = F.col(col_name)
    if target_type == "boolean":
        return F.when(F.lower(c).isin("true", "yes", "1"), F.lit(True)) \
                .when(F.lower(c).isin("false", "no", "0"), F.lit(False)) \
                .otherwise(F.lit(None)).alias(col_name)
    elif target_type == "long":
        return c.cast(LongType()).alias(col_name)
    elif target_type == "double":
        return c.cast(DoubleType()).alias(col_name)
    elif target_type == "date":
        return F.coalesce(F.to_date(c, "yyyy-MM-dd"), F.to_date(c, "MM/dd/yyyy")).alias(col_name)
    elif target_type == "timestamp":
        return F.coalesce(F.to_timestamp(c, "yyyy-MM-dd HH:mm:ss"),
                           F.to_timestamp(c, "yyyy-MM-dd'T'HH:mm:ss")).alias(col_name)
    else:
        # string: standardize by trimming whitespace
        return F.trim(c).alias(col_name)

# -------- VOID/NullType fix: cast all-null cols to explicit types --------
def fix_void_columns(df, final_schema_map=None):
    exprs = []
    for f in df.schema.fields:
        if isinstance(f.dataType, NullType):
            # If the column exists in final, cast to its type; else cast to STRING
            target_dt = final_schema_map.get(f.name, StringType()) if final_schema_map else StringType()
            exprs.append(F.col(f.name).cast(target_dt).alias(f.name))
        else:
            exprs.append(F.col(f.name))
    return df.select(*exprs)


StatementMeta(, bad056f4-0aa5-4743-9663-648699cc5fa4, 13, Finished, Available, Finished)

**7.0 This code retrieves daily Azure cost data for the current month—filtered specifically for Microsoft Fabric—by calling the Azure Cost Management API using client-credential authentication.**

It formats and normalizes the returned cost fields, converts the data into a clean table, and writes it to a Delta table when Spark is available (otherwise it returns a Pandas DataFrame).

In [12]:

import requests
import pandas as pd
from datetime import datetime, timedelta, timezone
from calendar import monthrange

# Optional: detect Spark context if you're running in a Fabric/Spark notebook
try:
    spark  # noqa: F821
    _HAS_SPARK = True
except NameError:
    _HAS_SPARK = False


def _aad_client_credentials_token(tenant_id: str, client_id: str, client_secret: str) -> str:
    """
    Acquire an AAD access token using OAuth2 client-credentials (v2 endpoint)
    for Azure Resource Manager (Cost Management) APIs.
    """
    token_url = f"https://login.microsoftonline.com/{tenant_id}/oauth2/v2.0/token"
    data = {
        "grant_type": "client_credentials",
        "client_id": client_id,
        "client_secret": client_secret,
        # v2 requires 'scope', not 'resource'
        "scope": "https://management.azure.com/.default",
    }
    r = requests.post(token_url, data=data, timeout=60)
    try:
        r.raise_for_status()
    except requests.HTTPError as ex:
        raise RuntimeError(f"Token request failed: {r.status_code} {r.text}") from ex
    return r.json()["access_token"]


def _month_bounds_utc(day: datetime) -> tuple[str, str]:
    """
    Return ISO 8601 timestamps (UTC) for the first and last moment of the month of 'day'.
    Example: ('2026-01-01T00:00:00Z', '2026-01-31T23:59:59Z')
    """
    year, month = day.year, day.month
    last_day = monthrange(year, month)[1]
    start_iso = datetime(year, month, 1, 0, 0, 0, tzinfo=timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ")
    end_iso   = datetime(year, month, last_day, 23, 59, 59, tzinfo=timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ")
    return start_iso, end_iso


def query_fabric_cost_daily_to_delta(
    tenant_id: str,
    client_id: str,
    client_secret: str,
    subscription_id: str,
    *,
    service_name_filter: str = "Microsoft Fabric",
    api_version: str = "2019-11-01",
    delta_table_name: str = "stg_fabric_cost_daily"
):
    """
    Queries Azure Cost Management for the current month's daily cost filtered by ServiceName,
    expands columns using response metadata, and writes to a Delta table if Spark is present.

    Returns:
        - If Spark available: writes to Delta and returns the Spark DataFrame.
        - Otherwise: returns a Pandas DataFrame.
    """

    # 1) Token
    access_token = _aad_client_credentials_token(tenant_id, client_id, client_secret)
    headers = {"Authorization": f"Bearer {access_token}", "Content-Type": "application/json"}

    # 2) Time range: 'last date from the cost table until yesterday'
    start_iso = get_max_or_default("fabric_cost_daily", "date")
    start_dt_utc = datetime.strptime(start_iso, "%Y-%m-%d").replace(tzinfo=timezone.utc)
    start_dt_utc = start_dt_utc.strftime("%Y-%m-%dT%H:%M:%SZ")

    current_dt_utc = (datetime.utcnow()).replace(tzinfo=timezone.utc)
    _, end_dt_utc = _month_bounds_utc(current_dt_utc) 

    
    start_dt_utc = datetime.fromisoformat(start_dt_utc.replace("Z", "+00:00"))
    end_dt_utc   = datetime.fromisoformat(end_dt_utc.replace("Z", "+00:00"))

    one_year = timedelta(days=180)

    if (end_dt_utc - start_dt_utc) > one_year:
        start_dt_utc = end_dt_utc - one_year

    # Only now, format to the timestamps you need downstream
    start_dt_iso = start_dt_utc.strftime("%Y-%m-%dT%H:%M:%SZ")
    end_dt_iso   = end_dt_utc.strftime("%Y-%m-%dT%H:%M:%SZ")

    print("Cost Data")
    print("Start Date:", start_dt_iso)
    print("End Date:", end_dt_iso)

    # 3) Cost Management Query endpoint (subscription scope)
    usage_url = (
        f"https://management.azure.com/subscriptions/{subscription_id}"
        f"/providers/Microsoft.CostManagement/query?api-version={api_version}"
    )

    # 4) Query body (keep your aggregations, groupings & filter)
    usage_data = {
        "type": "Usage",
        "timeframe": "Custom",
        "timePeriod": {"from": start_dt_iso, "to": end_dt_iso},
        "dataset": {
            "granularity": "Daily",
            "aggregation": {
                "totalCost": {"name": "Cost", "function": "Sum"}
            },
            "grouping": [
                {"type": "Dimension", "name": "ServiceName"},
                {"type": "Dimension", "name": "ResourceId"}
            ],
            "filter": {
                "dimensions": {
                    "name": "ServiceName",
                    "operator": "In",
                    "values": [service_name_filter]
                }
            }
        }
    }

    # 5) POST query
    r = requests.post(usage_url, headers=headers, json=usage_data, timeout=120)
    try:
        r.raise_for_status()
    except requests.HTTPError as ex:
        raise RuntimeError(f"Cost query failed: {r.status_code} {r.text}") from ex

    payload = r.json()
    # The response typically includes: properties.columns and properties.rows
    props = payload.get("properties", {})
    columns_meta = props.get("columns", [])
    rows = props.get("rows", [])

    if not columns_meta or not rows:
        # Nothing returned—return an empty DF to keep caller logic simple
        empty_df = pd.DataFrame(columns=[c.get("name", f"col_{i}") for i, c in enumerate(columns_meta)])
        if _HAS_SPARK:
            sdf = spark.createDataFrame(empty_df)  # noqa: F821
            sdf.write.mode("overwrite").format("delta").saveAsTable(delta_table_name)
            return sdf
        return empty_df

    # 6) Build columns from metadata (prevents column-order bugs)
    col_names = [c.get("name", f"col_{i}") for i, c in enumerate(columns_meta)]
    df = pd.DataFrame(rows, columns=col_names)

    # Your original mapping was:
    # cost, date, devicename, resourceid, currency
    # But Cost Mgmt columns vary with aggregation & groupings;
    # we keep dynamic names, then derive helpful fields.

    # Common columns expected from this query:
    # 'Cost', 'UsageDate', 'ServiceName', 'ResourceId', 'Currency'
    # Normalize where possible:
    # Cost
    if "Cost" in df.columns:
        df["cost"] = df["Cost"]
    # UsageDate (int YYYYMMDD) or string—normalize to datetime
    usage_date_col = next((c for c in df.columns if c.lower() in ("usagedate", "date")), None)
    if usage_date_col:
        df["date"] = pd.to_datetime(df[usage_date_col].astype(str), format="%Y%m%d", errors="coerce")
    else:
        # fallback to the time window start for all rows if not present
        df["date"] = pd.to_datetime(start_iso)

    # ResourceId → derive capacityname
    if "ResourceId" in df.columns:
        df["resourceid"] = df["ResourceId"]
        df["capacityname"] = df["ResourceId"].apply(
            lambda x: x.split("/capacities/")[-1] if isinstance(x, str) and "/capacities/" in x else None
        )
    else:
        df["resourceid"] = None
        df["capacityname"] = None

    # ServiceName → devicename (keep your original target field name)
    if "ServiceName" in df.columns:
        df["devicename"] = df["ServiceName"]
    else:
        df["devicename"] = service_name_filter

    # Currency
    if "Currency" in df.columns:
        df["currency"] = df["Currency"]
    elif "BillingCurrency" in df.columns:
        df["currency"] = df["BillingCurrency"]
    else:
        df["currency"] = None

    # Final projection matching your original table intent
    final_cols = ["cost", "date", "devicename", "resourceid", "currency", "capacityname"]
    for c in final_cols:
        if c not in df.columns:
            df[c] = None
    df = df[final_cols]

    # 7) Write to Delta if Spark available; else return pandas DF
    if _HAS_SPARK:
        sdf = spark.createDataFrame(df)  # noqa: F821
        sdf.write.mode("overwrite").format("delta").saveAsTable(delta_table_name)
        return sdf

    return df

StatementMeta(, bad056f4-0aa5-4743-9663-648699cc5fa4, 14, Finished, Available, Finished)

**8.0 This code reads each stg_ table, fixes null/void columns, infers and casts column types from a sample, and deduplicates rows (keeping the latest per key when an order column is provided).**

It then ensures a corresponding target table exists and MERGEs (upserts) the cleaned data into it using specified keys, optionally collecting record counts.
Finally, it returns a concise summary per table with keys, inferred types, void columns handled, and optional counts.

In [13]:

from pyspark.sql import functions as F
from pyspark.sql import Window
from pyspark.sql.types import StringType, NullType

def process_stage_tables(
    *,
    spark_session=None,
    stage_tables=None,
    pk_map=None,
    latest_order_col_map=None,
    dedupe_partition_key_map=None,
    merge_keys_map=None,
    sample_size: int = 1000,
    db: str | None = None,
    create_missing_final: bool = True,
    set_confs: bool = True,
    collect_counts: bool = False,   # counts can be expensive; default False
    log_prefix: str = "INGEST"
):
    """
    Ingest pattern: stage_* -> infer -> cast -> DEDUPE -> MERGE to final (Spark-only).

    Parameters
    ----------
    spark_session : SparkSession, optional
        Provide an explicit SparkSession; otherwise uses global 'spark'.
    stage_tables : list[str]
        List of stage tables to process (e.g., ["stg_items", "stg_dates", ...]).
    pk_map : dict[str, list[str]]
        Map of table -> primary keys (physical or business keys present in both stage & final).
    latest_order_col_map : dict[str, str], optional
        Table -> column name that defines “latest” (DESC). If missing/None, we dropDuplicates by dedupe keys.
    dedupe_partition_key_map : dict[str, list[str]], optional
        Table -> partition keys for dedupe (defaults to pk_map[table] if not specified).
        Use this when your physical PK includes time but you want to dedupe on a business key.
    merge_keys_map : dict[str, list[str]], optional
        Table -> keys used in MERGE ON (defaults to pk_map[table] if not specified).
        Use this to upsert on a subset of columns (e.g., unique_key only).
    sample_size : int
        Number of rows to sample for string type inference.
    db : str | None
        Optional database/schema name; if provided, tables are read/written as f"{db}.{table}".
    create_missing_final : bool
        If a final table doesn't exist, create it from the deduped stage DF.
    set_confs : bool
        If True, sets configs to keep auto-merge OFF and use legacy time parser.
    collect_counts : bool
        If True, collect counts for raw/deduped records in the summary (may be costly).
    log_prefix : str
        Prefix used in prints for readability.

    Returns
    -------
    list[dict]
        A summary per processed table with keys: table, final_table, merge_keys, void_cols, inferred_types, counts (optional).
    """

    # ----------------------------------------------------
    # 0) Resolve SparkSession and defaults
    # ----------------------------------------------------
    ss = spark_session or spark  # noqa: F821

    stage_tables = stage_tables or []
    pk_map = pk_map or {}
    latest_order_col_map = latest_order_col_map or {}
    dedupe_partition_key_map = dedupe_partition_key_map or {}
    merge_keys_map = merge_keys_map or {}

    if set_confs:
        # Keep auto-merge OFF; explicitly control schema/columns for MERGE
        ss.conf.set("spark.databricks.delta.schema.autoMerge.enabled", "false")
        # Use legacy parser to avoid strict parse errors (matches your original)
        ss.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

    # ----------------------------------------------------
    # 1) Helpers (Spark-only)
    # ----------------------------------------------------
    TRUE_SET  = ("true", "yes", "1")
    FALSE_SET = ("false", "no", "0")
    BOOL_SET  = TRUE_SET + FALSE_SET

    def infer_type_for_string_col(df_sample, col_name):
        """
        Infer a type for a STRING column using Spark-only ops on df_sample.
        Returns: 'boolean' | 'long' | 'double' | 'timestamp' | 'date' | 'string'.
        Rule: ALL non-empty values must satisfy the target pattern.
        """
        c = F.col(col_name)
        c_trim  = F.trim(c)
        c_lower = F.lower(c_trim)

        non_empty_cnt = df_sample.select(
            F.sum(F.when(c.isNotNull() & (c_trim != ""), 1).otherwise(0)).alias("non_empty")
        ).collect()[0]["non_empty"]
        if not non_empty_cnt or non_empty_cnt == 0:
            return "string"

        bool_cnt = df_sample.select(
            F.sum(F.when(c.isNotNull() & (c_trim != "") & c_lower.isin(*BOOL_SET), 1).otherwise(0)).alias("b")
        ).collect()[0]["b"]
        if bool_cnt == non_empty_cnt:
            return "boolean"

        int_cnt = df_sample.select(
            F.sum(F.when(c.isNotNull() & (c_trim != "") & c_trim.rlike(r"^-?\d+$"), 1).otherwise(0)).alias("i")
        ).collect()[0]["i"]
        if int_cnt == non_empty_cnt:
            return "long"

        float_cnt = df_sample.select(
            F.sum(F.when(c.isNotNull() & (c_trim != "") & c_trim.rlike(r"^-?\d+(?:\.\d+)?$"), 1).otherwise(0)).alias("f")
        ).collect()[0]["f"]
        if float_cnt == non_empty_cnt:
            return "double"

        ts_parsed = F.coalesce(
            F.to_timestamp(c_trim, "yyyy-MM-dd HH:mm:ss"),
            F.to_timestamp(c_trim, "yyyy-MM-dd'T'HH:mm:ss"),
            F.to_timestamp(c_trim, "MM/dd/yyyy HH:mm:ss")
        )
        ts_cnt = df_sample.select(
            F.sum(F.when(c.isNotNull() & (c_trim != "") & ts_parsed.isNotNull(), 1).otherwise(0)).alias("t")
        ).collect()[0]["t"]
        if ts_cnt == non_empty_cnt:
            return "timestamp"

        d_parsed = F.coalesce(
            F.to_date(c_trim, "yyyy-MM-dd"),
            F.to_date(c_trim, "MM/dd/yyyy"),
            F.to_date(c_trim, "yyyy/MM/dd")
        )
        d_cnt = df_sample.select(
            F.sum(F.when(c.isNotNull() & (c_trim != "") & d_parsed.isNotNull(), 1).otherwise(0)).alias("d")
        ).collect()[0]["d"]
        if d_cnt == non_empty_cnt:
            return "date"

        return "string"

    def cast_column(col_name, target_type):
        """Apply casting logic to a column; always explicit for STRING."""
        c = F.col(col_name)
        c_trim = F.trim(c)

        if target_type == "boolean":
            return (
                F.when(F.lower(c_trim).isin(*TRUE_SET),  F.lit(True))
                 .when(F.lower(c_trim).isin(*FALSE_SET), F.lit(False))
                 .otherwise(F.lit(None))
            ).alias(col_name)

        if target_type == "long":
            return c.cast("long").alias(col_name)

        if target_type == "double":
            return c.cast("double").alias(col_name)

        if target_type == "date":
            return F.coalesce(
                F.to_date(c, "yyyy-MM-dd"),
                F.to_date(c, "MM/dd/yyyy"),
                F.to_date(c, "yyyy/MM/dd")
            ).alias(col_name)

        if target_type == "timestamp":
            return F.coalesce(
                F.to_timestamp(c, "yyyy-MM-dd HH:mm:ss"),
                F.to_timestamp(c, "yyyy-MM-dd'T'HH:mm:ss"),
                F.to_timestamp(c, "MM/dd/yyyy HH:mm:ss")
            ).alias(col_name)

        # explicit string
        return F.trim(c).cast(StringType()).alias(col_name)

    def keep_latest_per_key(df, partition_keys, order_col):
        """
        Deduplicate df by partition_keys; keep latest row by order_col DESC.
        If order_col not provided/absent, falls back to dropDuplicates(partition_keys).
        """
        if not order_col or order_col not in df.columns:
            print(f"⚠️  [{log_prefix}] No valid 'latest' column; dropDuplicates({partition_keys}).")
            return df.dropDuplicates(partition_keys)

        w = Window.partitionBy([F.col(c) for c in partition_keys]).orderBy(F.col(order_col).desc())
        return (
            df.withColumn("_rn", F.row_number().over(w))
              .filter(F.col("_rn") == 1)
              .drop("_rn")
        )
    
    
    def table_exists(fully_qualified_name: str) -> bool:
        """
        Accepts 1-, 2-, or 3-part names:
        - "table"
        - "db.table"
        - "catalog.schema.table" (Unity Catalog / Fabric)
        """
        try:
            return bool(spark.catalog.tableExists(fully_qualified_name))
        except Exception:
            # Defensive: tableExists usually won't throw, but be safe.
            return False

    # ----------------------------------------------------
    # 2) Main loop
    # ----------------------------------------------------
    summaries = []

    for stg in (stage_tables or []):
        print(f"\n=== [{log_prefix}] Processing: {stg} ===")

        
        if not table_exists(stg):
            print(f"⚠️ Skipping '{stg}': table does not exist.")
            continue
    
        if stg not in pk_map:
            raise ValueError(f"[{log_prefix}] Missing PK in pk_map for {stg}")

        final_table = stg.replace("stg_", "")
        pk_cols     = pk_map[stg]
        part_keys   = dedupe_partition_key_map.get(stg, pk_cols)
        order_col   = latest_order_col_map.get(stg)
        merge_keys  = merge_keys_map.get(stg, pk_cols)

        # A) Read stage
        stage_fqn = f"{db}.{stg}" if db else stg
        df_raw = ss.read.table(stage_fqn)

        # Force void/NullType to STRING
        void_cols = [
            f.name for f in df_raw.schema.fields
            if isinstance(f.dataType, NullType) or f.dataType.simpleString().lower() in ("void", "null")
        ]
        if void_cols:
            print(f"[{log_prefix}] Force-casting void columns to STRING: {void_cols}")

        df_full = df_raw.select([
            F.trim(F.col(c)).cast(StringType()).alias(c) if c in void_cols else F.col(c).alias(c)
            for c in df_raw.columns
        ])

        # B) Infer types for STRING columns on sample
        df_sample = df_full.limit(sample_size)
        string_cols = [f.name for f in df_sample.schema.fields if f.dataType.simpleString() == "string"]

        inferred = {}
        for c in string_cols:
            inferred[c] = infer_type_for_string_col(df_sample, c)
        print(f"[{log_prefix}] Inferred string types: {inferred}")

        # C) Cast full DF
        df_cast = df_full.select([
            cast_column(c, inferred[c]) if c in inferred else F.col(c).alias(c)
            for c in df_full.columns
        ])

        print(f"[{log_prefix}] Stage schema (no void expected):")

        # df_cast.printSchema()

        # D) DEDUPE
        # safety checks
        for k in part_keys:
            if k not in df_cast.columns:
                raise ValueError(f"[{log_prefix}] Partition key '{k}' missing in {stg}; cols: {df_cast.columns}")
        if order_col and order_col not in df_cast.columns:
            raise ValueError(f"[{log_prefix}] Order-by column '{order_col}' missing in {stg}; cols: {df_cast.columns}")

        df_latest = keep_latest_per_key(df_cast, part_keys, order_col)

        # E) Ensure final exists
        final_fqn = f"{db}.{final_table}" if db else final_table
        if not ss.catalog.tableExists(final_fqn):
            if create_missing_final:
                print(f"[{log_prefix}] Creating final table: {final_fqn}")
                df_latest.write.mode("overwrite").saveAsTable(final_fqn)
            else:
                raise ValueError(f"[{log_prefix}] Final table does not exist: {final_fqn}")

        # F) MERGE only common columns
        final_df   = ss.read.table(final_fqn)
        final_cols = final_df.columns
        common     = [c for c in df_latest.columns if c in final_cols]

        if not common:
            print(f"[{log_prefix}] WARNING: No common columns between {stg} and {final_fqn}; skipping.")
            summaries.append({
                "table": stg, "final_table": final_fqn, "merge_keys": merge_keys,
                "void_cols": void_cols, "inferred_types": inferred, "skipped": True
            })
            continue

        # Temp views must be unique per table to avoid clashes
        tv_final = f"final_view_{final_table}"
        tv_stage = f"stage_view_{final_table}"
        final_df.select(common).createOrReplaceTempView(tv_final)
        df_latest.select(common).createOrReplaceTempView(tv_stage)

        # ON clause
        on_pairs = [f"f.`{pk}` = s.`{pk}`" for pk in merge_keys if pk in common]
        missing_pks = [pk for pk in merge_keys if pk not in common]
        if missing_pks:
            raise ValueError(f"[{log_prefix}] MERGE key(s) {missing_pks} not in common columns for {stg} -> {final_fqn}")

        on_clause = " AND ".join(on_pairs)
        non_pk    = [c for c in common if c not in merge_keys]
        set_clause  = ", ".join([f"f.`{c}` = s.`{c}`" for c in non_pk]) if non_pk else ""
        insert_cols = ", ".join([f"`{c}`" for c in common])
        insert_vals = ", ".join([f"s.`{c}`" for c in common])

        merge_sql = f"""
        MERGE INTO {final_fqn} AS f
        USING {tv_stage} AS s
        ON {on_clause}
        {"WHEN MATCHED THEN UPDATE SET " + set_clause if set_clause else ""}
        WHEN NOT MATCHED THEN INSERT ({insert_cols}) VALUES ({insert_vals})
        """

        # print(merge_sql)
        ss.sql(merge_sql)

        print(f"[{log_prefix}] Upserted {stg} -> {final_fqn} on keys {merge_keys}")

        # Optional counts (actions)
        counts = {}
        if collect_counts:
            try:
                counts["stage_raw"]   = df_raw.count()
                counts["stage_latest"] = df_latest.count()
                counts["final_after"] = ss.read.table(final_fqn).count()
            except Exception as _:
                counts["error"] = "count_failed"

        summaries.append({
            "table": stg,
            "final_table": final_fqn,
            "merge_keys": merge_keys,
            "void_cols": void_cols,
            "inferred_types": inferred,
            "counts": counts if counts else None
        })

    return summaries


StatementMeta(, bad056f4-0aa5-4743-9663-648699cc5fa4, 15, Finished, Available, Finished)

**9.0 This function runs the full ingestion pipeline by defining all stage tables and their keys, then calling process_stage_tables() to clean, dedupe, type-cast, and merge each stage table into its final table.**

After processing, it prints a summary of what happened for each table.

In [14]:
def merge_stage_data():

        # 0) Your configuration (same shape as in your notebook)
        stage_tables = [
            "stg_capacities",
            "stg_dates",
            "stg_workloads",
            "stg_workspaces",
            "stg_metrics_by_item_and_day",
            "stg_items",
            "stg_fabric_cost_daily"
        ]

        pk_map = {
            "stg_capacities": ["capacities_capacity_id"],
            "stg_dates": ["dates_date"],
            "stg_workloads": ["workloads_workload_kind"],
            "stg_workspaces": ["workspaces_workspace_id"],
            "stg_metrics_by_item_and_day": ["metrics_by_item_and_day_unique_key", "metrics_by_item_and_day_datetime"],
            "stg_items": ["items_item_id"],
            "stg_fabric_cost_daily": ["date", "capacityname"],
        }

        latest_order_col_map = {
            # "stg_metrics_by_item_and_day": "metrics_by_item_and_day_datetime",
            "stg_items": "items_timestamp",
        }

        dedupe_partition_key_map = {
            # "stg_metrics_by_item_and_day": ["metrics_by_item_and_day_unique_key"],
        }

        merge_keys_map = {
            # "stg_metrics_by_item_and_day": ["mstg_metrics_by_item_and_day$0costetrics_by_item_and_day_unique_key"],
        }

        # 1) Run it
        summary = process_stage_tables(
            stage_tables=stage_tables,
            pk_map=pk_map,
            latest_order_col_map=latest_order_col_map,
            dedupe_partition_key_map=dedupe_partition_key_map,
            merge_keys_map=merge_keys_map,
            sample_size=1000,
            db=None,                     # or your database/schema name
            create_missing_final=True,
            set_confs=True,
            collect_counts=False,        # enable if you need counts
            log_prefix="PIPELINE"
        )

        # 2) Inspect result
        for s in summary:
            print(s)

StatementMeta(, bad056f4-0aa5-4743-9663-648699cc5fa4, 16, Finished, Available, Finished)

**Main workflow starts here**

1. Get the service principal details from the Key Vault.
2. Get the Metrics app workspace details and the dataset details.
3. Extract Fabric cost data.
4. For each capacity in the tenant, extract and process the capacity metrics data.

In [15]:
# Extracts tenantid, subscriptionid, appid,appsecret from the Key Vault
secrets  = get_kv_secrets(
    kv_name="kvrbacpocsk",
    secret_names=["tenantid", "subscriptionid", "appid", "appsecret"]
)

tenant_id = secrets["tenantid"]
subscriptionid = secrets["subscriptionid"]
client_id = secrets["appid"]
client_secret = secrets["appsecret"]

StatementMeta(, bad056f4-0aa5-4743-9663-648699cc5fa4, 17, Finished, Available, Finished)

In [16]:
# Resolve Capcity Metrics workspace details
# All functions now reuse the same fabric context
metrics_ws_name, metrics_ws_id, metrics_ds_name, metrics_ds_id = extract_metrics_workspace_details()
print(metrics_ws_name, metrics_ws_id, metrics_ds_name, metrics_ds_id)

StatementMeta(, bad056f4-0aa5-4743-9663-648699cc5fa4, 18, Finished, Available, Finished)

Microsoft Fabric Capacity Metrics 4bdee50b-585a-489a-bf6a-13ad81fdaaa4 Fabric Capacity Metrics 54f74ce5-da8c-4d87-91b3-f6550234856a


In [17]:
# Extract Fabric cost data
df_or_sdf = query_fabric_cost_daily_to_delta(
    tenant_id=tenant_id,
    client_id=client_id,
    client_secret=client_secret,
    subscription_id=subscriptionid,
    service_name_filter="Microsoft Fabric",   # change if needed
    delta_table_name="stg_fabric_cost_daily"
)
# display(df_or_sdf)  # works for Spark DF; for pandas, print(df_or_sdf.head())

StatementMeta(, bad056f4-0aa5-4743-9663-648699cc5fa4, 19, Finished, Available, Finished)

Cost Data
Start Date: 2026-01-27T00:00:00Z
End Date: 2026-02-28T23:59:59Z


In [18]:
# Get all capcities in the tenant and for each capacity extarct the details from the metrics app into stage tables 
# and then merage the stage data into target tables

import sempy.fabric as fabric
from sempy.fabric.exceptions import FabricHTTPException
from notebookutils import notebook

with set_service_principal(tenant_id=tenant_id, client_id=client_id, client_secret=client_secret):
     client = fabric.FabricRestClient()
url  = f"https://api.powerbi.com/v1.0/myorg/capacities"
fabric_rest_client = fabric.FabricRestClient()
capacities_json = fabric_rest_client.get(url)
# get a list of all capacities
capacities = capacities_json.json()["value"]

# process data from each capacity
for cap in capacities:
    region_name = cap.get("region", "Unknown Region")
    capacity_id_to_extract_data_from = cap.get("id", "000000")
    
    print("######### processing data #############")
    print("processing the data for the capacity :", f"{capacity_id_to_extract_data_from}") 
    print("######### processing data #############")

    # update metrics dataset 
    update_metrics_dataset(metrics_workspace_id = metrics_ws_id, metrics_dataset_id = metrics_ds_id, capacity_id_to_extract_data_from = capacity_id_to_extract_data_from, region_name = region_name)

    # list of table to be processed
    table_list = [
            "Capacities",
            "Dates",
            "Metrics By Item and Day",
            "Workloads",
            "Workspaces",
            "Items",
            "Metrics By Item"
            ]
    for table_name in table_list:
            print("processing the table:", table_name)      
            # extract data for each one of the tables from metrics dataset
            get_table_data(metrics_workspace_id = metrics_ws_id, metrics_dataset_id = metrics_ds_id, table_name= table_name, capacity_id = capacity_id_to_extract_data_from)
    # merge stage data into target tables            
    merge_stage_data()

#     if bool(spark.catalog.tableExists("metrics_by_item_and_day")) and bool(spark.catalog.tableExists("stg_metrics_by_item_and_day")):
#        display(spark.sql("select distinct metrics_by_item_and_day_capacity_id FROM Metrics.metrics_by_item_and_day union select distinct metrics_by_item_and_day_capacity_id FROM Metrics.stg_metrics_by_item_and_day"))

StatementMeta(, bad056f4-0aa5-4743-9663-648699cc5fa4, 20, Submitted, Running, Running)

######### processing data #############
processing the data for the capacity : 6A3D303C-56B7-4D7C-869D-1FBFC4D539CB
######### processing data #############
✅ Parameters updated successfully.
🔄 Triggering dataset refresh…
[0] status=Unknown
[1] status=Unknown
[2] status=Unknown
[3] status=Unknown
[4] status=Completed
Refresh succeeded.
processing the table: Capacities
processing the table: Dates
processing the table: Metrics By Item and Day
getting the data from start_iso = 2026-01-28
An unexpected error occurred: list index out of range
processing the table: Workloads
processing the table: Workspaces
processing the table: Items
processing the table: Metrics By Item
An unexpected error occurred: list index out of range

=== [PIPELINE] Processing: stg_capacities ===
[PIPELINE] Inferred string types: {'capacities_capacity_id': 'string', 'capacities_state': 'string', 'capacities_region_without_default': 'string', 'capacities_capacity_name': 'string', 'capacities_sku': 'string', 'capacities